**Benchmark General Information Loader (BGI)**

WHAT IT IS:

- This script manages the creation and population of the BenchmarkGeneralInformation table

- Benchmarks represent market indexes used to evaluate portfolio performance (e.g., VNQ, IYR, SCHH)


WHAT IT DOES:

- Defines benchmark metadata (code, name, symbol, etc.) in a structured DataFrame
- Validates the schema and field types to ensure data consistency
- Uploads the data to a staging table in Snowflake
- Performs a MERGE operation to insert new benchmarks or update existing ones

WHY IT'S IMPORTANT:

- Benchmarks are essential for measuring portfolio returns against relevant market indices
- A clean and consistent BGI table allows for accurate performance attribution, reporting, and analysis
- The script is designed to be scalable, repeatable, and easy to extend with additional benchmarks

In [ ]:
!pip install yfinance

# **Benchmark General Information Table**


This table is meant to be static and can be updated via CSV. There is a proposed yaml file as an alternative to increase scalability, but it needs improvement.

## New Code

In [ ]:
from pickle import FALSE  # Unused import – can be removed safely
import pandas as pd

# Define static benchmark metadata entries
# Each dictionary represents a single benchmark and its key attributes
bgi_data = [
    {
        "BENCHMARKCODE": "REITBENCH01",
        "NAME": "Vanguard Real Estate Index Fund ETF",
        "SYMBOL": "VNQ",
        "ISBEGINOFDAYPERFORMANCE": False
    },
    {
        "BENCHMARKCODE": "REITBENCH02",
        "NAME": "iShares US Real Estate ETF",
        "SYMBOL": "IYR",
        "ISBEGINOFDAYPERFORMANCE": False
    },
    {
        "BENCHMARKCODE": "REITBENCH03",
        "NAME": "Schwab US REIT ETF",
        "SYMBOL": "SCHH",
        "ISBEGINOFDAYPERFORMANCE": False
    }
]

# Convert the list of benchmark dictionaries into a pandas DataFrame
bgi_df = pd.DataFrame(bgi_data)

# Output the final structured benchmark information
print(bgi_df)


  BENCHMARKCODE                                 NAME SYMBOL  \
0   REITBENCH01  Vanguard Real Estate Index Fund ETF    VNQ   
1   REITBENCH02           iShares US Real Estate ETF    IYR   
2   REITBENCH03                   Schwab US REIT ETF   SCHH   

   ISBEGINOFDAYPERFORMANCE  
0                    False  
1                    False  
2                    False  


## Validation

In [ ]:
def assert_bgi(df):
    """
    Validates the structure and integrity of a Benchmark General Information (BGI) DataFrame.

    This function performs the following checks:
    1. Ensures that all required columns are present: 'BENCHMARKCODE', 'NAME', 'SYMBOL', 'ISBEGINOFDAYPERFORMANCE'.
    2. Ensures that the critical fields ('BENCHMARKCODE', 'NAME', 'SYMBOL') do not contain null values.
    3. Ensures that the 'ISBEGINOFDAYPERFORMANCE' column is of boolean dtype.

    Parameters:
        df (pd.DataFrame): The BGI DataFrame to validate.

    Raises:
        ValueError: If any required columns are missing or if null values are found in key fields.
        TypeError: If 'ISBEGINOFDAYPERFORMANCE' is not of boolean type.

    Returns:
        None. Prints confirmation if validation passes.
    """
    required_columns = {"BENCHMARKCODE", "NAME", "SYMBOL", "ISBEGINOFDAYPERFORMANCE"}
    actual_columns = set(df.columns)

    # Check all required columns exist
    missing = required_columns - actual_columns
    if missing:
        raise ValueError(f"Missing required BGI columns: {missing}")

    # Check for nulls in key fields
    nulls = df[["BENCHMARKCODE", "NAME", "SYMBOL"]].isnull().any()
    for col, is_null in nulls.items():
        if is_null:
            raise ValueError(f"Null values found in column: {col}")

    # Check boolean type
    if not pd.api.types.is_bool_dtype(df["ISBEGINOFDAYPERFORMANCE"]):
        raise TypeError("ISBEGINOFDAYPERFORMANCE must be boolean")

    print("BGI validation passed.")


## yaml version

In [2]:
import yaml
import pandas as pd

def load_benchmark_config(yaml_path: str) -> pd.DataFrame:
    """
    Loads benchmark metadata from a YAML configuration file and returns a pandas DataFrame.

    Parameters:
        yaml_path (str): Path to the YAML file containing benchmark definitions.

    Returns:
        pd.DataFrame: DataFrame with benchmark information, including BENCHMARKCODE, NAME,
                      SYMBOL, and ISBEGINOFDAYPERFORMANCE.
    """
    with open(yaml_path, "r") as f:
        # Load list of benchmark dicts from the YAML file
        bgi_data = yaml.safe_load(f)

    # Convert list of dictionaries to a DataFrame
    return pd.DataFrame(bgi_data)

# Example usage:
# bgi_df = load_benchmark_config("benchmarks.yaml")
# print(bgi_df)


# **Connecting to Snowflake**

## Establishing Connection

You must download the env.txt file which is located in Github (env.txt) and upload using the "Choose Files" widget after running the below cell.

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving env.txt to env.txt


In [ ]:
import os

if os.path.exists("env.txt"):
    os.rename("env.txt", "env")
    print("Renamed env.txt to env")
else:
    print("File not found. Make sure you uploaded env.txt.")

Renamed env.txt to env


In [ ]:
!pip install snowflake-connector-python python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 6.1 MB/s eta 0:00:00


In [ ]:
from dotenv import load_dotenv
load_dotenv("env")  # This loads the Snowflake credentials into the environment

True

In [ ]:
import snowflake.connector
import os

# Pull in credentials
conn = snowflake.connector.connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database=os.getenv("SNOWFLAKE_DATABASE"),
    schema=os.getenv("SNOWFLAKE_SCHEMA")
)

# Run a simple test query
cur = conn.cursor()
cur.execute("SELECT CURRENT_USER(), CURRENT_ROLE(), CURRENT_DATABASE();")
for row in cur:
    print(row)

cur.close()
conn.close()

('SHIVNEETN', 'AST_REALESTATE_DB_RW', 'AST_REALESTATE_DB')


## Populating BGI in Snowflake

In [ ]:
from snowflake.connector import connect
from snowflake.connector.pandas_tools import write_pandas
import os
import pandas as pd

# Optional: run validation first
# from assert_bgi import assert_bgi
# assert_bgi(bgi_df)

# Connect to Snowflake
conn = connect(
    user=os.getenv("SNOWFLAKE_USER"),
    password=os.getenv("SNOWFLAKE_PASSWORD"),
    account=os.getenv("SNOWFLAKE_ACCOUNT"),
    role=os.getenv("SNOWFLAKE_ROLE"),
    warehouse=os.getenv("SNOWFLAKE_WAREHOUSE"),
    database="AST_REALESTATE_DB",
    schema="DBO"
)

# Ensure correct schema
with conn.cursor() as cur:
    cur.execute("USE DATABASE AST_REALESTATE_DB")
    cur.execute("USE SCHEMA DBO")

    # Create staging table (temporary or permanent depending on your process)
    cur.execute("""
        CREATE OR REPLACE TEMP TABLE STG_BENCHMARKGENERALINFORMATION (
            BENCHMARKCODE VARCHAR,
            NAME VARCHAR,
            SYMBOL VARCHAR,
            ISBEGINOFDAYPERFORMANCE BOOLEAN
        )
    """)

# Upload to staging
success, nchunks, nrows, _ = write_pandas(
    conn, df=bgi_df, table_name="STG_BENCHMARKGENERALINFORMATION", quote_identifiers=False
)
print(f"Uploaded {nrows} rows to STG_BENCHMARKGENERALINFORMATION")

# MERGE into target table
with conn.cursor() as cur:
    cur.execute("""
        MERGE INTO BenchmarkGeneralInformation AS target
        USING STG_BENCHMARKGENERALINFORMATION AS source
        ON target.BENCHMARKCODE = source.BENCHMARKCODE

        WHEN MATCHED THEN UPDATE SET
            target.NAME = source.NAME,
            target.SYMBOL = source.SYMBOL,
            target.ISBEGINOFDAYPERFORMANCE = source.ISBEGINOFDAYPERFORMANCE

        WHEN NOT MATCHED THEN INSERT (
            BENCHMARKCODE, NAME, SYMBOL, ISBEGINOFDAYPERFORMANCE
        ) VALUES (
            source.BENCHMARKCODE, source.NAME, source.SYMBOL, source.ISBEGINOFDAYPERFORMANCE
        )
    """)
    print("MERGE into BenchmarkGeneralInformation completed.")


Uploaded 3 rows to STG_BENCHMARKGENERALINFORMATION
MERGE into BenchmarkGeneralInformation completed.


# **Data Flow & Architecture Mapping**

## Source to Storage Mapping

| Stage                        | Description                                                                                             |
| ---------------------------- | ------------------------------------------------------------------------------------------------------- |
| **Source**                   | Manual inputs or internal static mapping of REIT benchmarks (`SYMBOL`, `NAME`, `BENCHMARKCODE`)         |
| **Extraction**               | Loaded into Python as `bgi_df` via Excel seed, Snowflake query, or hardcoded config                     |
| **Transformation**           | Minor cleaning to standardize `SYMBOL`, ensure valid tickers, and tag with `ISBEGINOFDAYPERFORMANCE`    |
|  • `symbol.strip()`          | Removes special characters like `$`, extra spaces for compatibility with Yahoo Finance                  |
|  • `name.title()`            | Normalizes benchmark names                                                                              |
|  • `map benchmark code`      | Assigns fixed codes: `REITBENCH001`, `REITBENCH002`, etc.                                               |
|  • `ISBEGINOFDAYPERFORMANCE` | Set to `True` to indicate NAV-style processing                                                          |
| **Validation**               | `assert_bgi()` or manual assertions validate required fields and naming patterns                        |
| **Storage**                  | Uploaded to `STG_BENCHMARKGENERALINFORMATION` → `MERGE`d into final `BenchmarkGeneralInformation` table |
| **Consumption**              | Used to drive benchmark universe, match to performance series, power UI dropdowns & audits              |


## Data Flow Diagram

          +--------------------------+
          | Local Seed (Excel/JSON) |
          +--------------------------+
                      |
          [pd.read_excel() or manual entry]
                      |
                      v
         +-------------------------------+
         | Clean + Format bgi_df         |
         | - symbol.strip()              |
         | - name.title()                |
         | - assign ISBEGINOFDAYPERFORMANCE |
         +-------------------------------+
                      |
                      v
         +-------------------------------+
         | Optional: assert_bgi()        |
         | - Check for missing values    |
         | - Valid symbol formatting     |
         +-------------------------------+
                      |
                      v
         +-------------------------------+
         | write_pandas() to staging     |
         | STG_BENCHMARKGENERALINFORMATION |
         +-------------------------------+
                      |
                      v
         +-------------------------------+
         | MERGE into BGI table          |
         +-------------------------------+
                      |
                      v
         +-------------------------------+
         | Used by benchmark engine, UI  |
         +-------------------------------+


## Transformation Logic Documentation

### Symbol

| Transformation | Description                                                           |
| -------------- | --------------------------------------------------------------------- |
| Strip prefix   | Removes `$` or whitespace from symbols (e.g., `"$VNQ"` → `"VNQ"`)     |
| Validation     | Ensures symbol is compatible with Yahoo Finance (`yf.Ticker(symbol)`) |
| Type           | `VARCHAR` — must be valid for external data pulls                     |


### BenchmarkCode

| Transformation | Description                                                         |
| -------------- | ------------------------------------------------------------------- |
| Manual mapping | Fixed assignments: `VNQ → REITBENCH001`, `IYR → REITBENCH002`, etc. |
| Uniqueness     | Primary key in final table                                          |
| Type           | `VARCHAR` — follows naming standard `REITBENCH###`                  |


### Name

| Transformation | Description                                  |
| -------------- | -------------------------------------------- |
| Title case     | Standardized (e.g., `"Vanguard REIT Index"`) |
| Validation     | Should match symbol’s ETF/fund description   |
| Type           | `VARCHAR`                                    |


### IsBeginOfDayPerformance

| Transformation | Description                                              |
| -------------- | -------------------------------------------------------- |
| Default value  | Always `True` for NAV-style or adjusted price series     |
| Purpose        | Tells downstream logic how to time-align with portfolios |
| Type           | `BOOLEAN`                                                |


### Validation

| Check                 | Description                                                                  |
| --------------------- | ---------------------------------------------------------------------------- |
| Required fields       | `BENCHMARKCODE`, `SYMBOL`, `NAME`, `ISBEGINOFDAYPERFORMANCE` must be present |
| Valid ticker format   | Symbols must resolve in `yfinance` or mapped correctly                       |
| Unique benchmark code | No duplicates allowed                                                        |
| Optional integration  | Could cross-validate symbols against a live ETF ticker list                  |


## Future Enhancements

| Feature               | Description                                                                    |
| --------------------- | ------------------------------------------------------------------------------ |
| Benchmark Category    | Add `CATEGORY` column (e.g., `REIT`, `Equity`, `Fixed Income`)                 |
| Inception Date        | Add date metadata for benchmark tracking start                                 |
| Override System       | Introduce manual flag for symbol suppression or rerouting                      |
| Symbol Health Monitor | Automated `yfinance` ping to validate ticker availability                      |
| Benchmark Type Enum   | Formalize NAV-based vs. price-based benchmarks (`Total Return`, `Price`, etc.) |
| CI Integration        | Add pre-merge validation in a CI pipeline                                      |
